In [20]:
import json
import pandas as pd
import numpy as np
import urllib3
from tqdm import trange
import os
import torch
from ultralytics import YOLO




In [21]:
metadata = pd.read_parquet("dataset/the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet")
print(metadata.shape)
metadata = metadata[0:20000]

(12933524, 8)


In [22]:
model = YOLO("yolov8n.pt")
IMG_FORMATS = ['bmp', 'dng', 'jpeg', 'jpg', 'mpo', 'png', 'tif', 'tiff', 'webp', 'pfm']
urls_raw = list(metadata["URL"])
caps_raw = list(metadata["TEXT"])

In [23]:
metadata[0:1]

,SAMPLE_ID,URL,TEXT,HEIGHT,WIDTH,LICENSE,NSFW,similarity
0,1.581282e+12,http://media.rightmove.co.uk/148k/147518/58718...,View EPC Rating Graph for this property,109.0,100.0,?,UNSURE,0.312813


In [31]:
import re
def cap_parse(cap):
    nums = []
    x = re.findall(r'\b(two|three|four|five|six|seven|eight|nine|ten)\b', cap, flags=re.IGNORECASE)
    num2word = {
        'two':2,
        'three':3,
        'four':4,
        'five':5,
        'six':6,
        'seven':7,
        'eight':8,
        'nine':9,
        'ten':10
    }
    if len(x) == 0 :return (False,-1)
    else:
        max_freq = max(num2word[i.lower()] for i in x)
        return (True,max_freq)
    

txt0 = "vintage silver plate tablespoons, serving spoon set of six 1847 Rogers pattern"
txt1 = "A preview of seven printable dot marker coloring pages. Each page has a large 4th of July themed image with dots to color in with a dauber style marker."
txt2 = "bag of chips"
cap_parse(txt2)


(False, -1)

In [32]:

def validate_urls(urls,caps):
    valid_urls = []
    valid_caps = []
    for url,cap in zip(urls,caps):
        for img_format in IMG_FORMATS:
            idx = url.lower().find("." + img_format)
            if idx == -1:
                continue
            valid_urls.append(url[:idx + len(img_format) + 1])
            valid_caps.append(cap)
    return (valid_urls,valid_caps)




In [33]:
def validate_counts(counting):
    final = []
    for i in range(len(counting)):
        x,cap,url,num = counting[i]
        if max(set(x)) == num:
            final.append(counting[i])    

In [34]:
def save_counts(counting):    
    print(len(counting))
    counting[0:5]
    f = open("segments0.npy","wb")
    np.save(f,pd.DataFrame(counting))
    print("counting set saved")


In [35]:

def filter_laion(urls_raw,caps_raw):
    urls,caps = validate_urls(urls_raw,caps_raw)
    counting = []
    for i in trange(2000):
        cap_data = cap_parse(caps[i])
        # print(cap_data,caps[i])
        if cap_data[0] == True:
            try:
                results = model(str(urls[i]),verbose=False,stream=False)
                x = list(results[0].boxes.cls)
            except:
                x = []
            counting.append((x,urls[i],caps[i],cap_data[1]))
    # counting = validate_counts(counting)
    
    return counting

        

In [36]:
filter_laion(urls_raw,caps_raw)

    

  0%|          | 0/2000 [00:00<?, ?it/s]

Found http://mediad.publicbroadcasting.net/p/nwpr/files/styles/card_280/public/201211/110912AK_Coldpress_3.jpg locally at 110912AK_Coldpress_3.jpg


  7%|▋         | 133/2000 [00:00<00:02, 690.99it/s]

Found http://www.pinkdiamondusa.com/media/catalog/product/cache/1/small_image/150x150/9df78eab33525d08d6e5fb8d27136e95/L/Q/LQ7829P_3.jpg locally at LQ7829P_3.jpg
Found https://static1.bigstockphoto.com/thumbs/7/3/1/large2/137946380.jpg locally at 137946380.jpg


 17%|█▋        | 333/2000 [00:00<00:02, 699.28it/s]

...

Found http://images.buyagift.co.uk/common/client/Images/Product/Small/en-GB/marriott_windsor_Exterior_G.jpg locally at marriott_windsor_Exterior_G.jpg


 94%|█████████▎| 1871/2000 [01:23<00:28,  4.50it/s]

Found https://assets.bwbx.io/images/users/iqjWHBFdfxIU/i_2b1hndCCpQ/v0/560x-1.jpg locally at 560x-1.jpg


100%|██████████| 2000/2000 [01:23<00:00, 24.02it/s]


[([tensor(39.),
   tensor(39.),
   tensor(39.),
   tensor(39.),
   tensor(39.),
   tensor(39.)],
  'http://mediad.publicbroadcasting.net/p/nwpr/files/styles/card_280/public/201211/110912AK_Coldpress_3.jpg',
  '"The Durant family makes two different kinds of regional olive oil. One called a ""fresh press"" is their minimally processed oil which yields a greener, spicier flavor. The other, is high-quality extra virgin."',
  2),
 ([],
  'http://www.pinkdiamondusa.com/media/catalog/product/cache/1/small_image/150x150/9df78eab33525d08d6e5fb8d27136e95/L/Q/LQ7829P_3.jpg',
  '18K Two-tone Gold Fancy Diamond Slice Pendant',
  2),
 ([tensor(33.)],
  'https://static1.bigstockphoto.com/thumbs/7/3/1/large2/137946380.jpg',
  'Vector illustration of a bouquet for you Image bouquet of black and white style of pointillism for you in the middle of drawing flowers flying around seven three butterflies on a gray background',
  7),
 ([tensor(39.), tensor(39.), tensor(39.)],
  'https://as1.ftcdn.net/jpg/01/

In [ ]:
f = open("segments0.npy","rb")
out = np.load(f,allow_pickle=True)

In [ ]:
import requests

def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   try:
      r = requests.head(image_url)
   except:
      return False
   try:
      if r.headers["content-type"] in image_formats:
         return True
   except:
         return True
      
   return False